<a href="https://colab.research.google.com/github/iamvarol/blogposts/blob/main/medium/europe_gas_network/european_gas_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Neo4j Sandbox Setup

In [ ]:
!pip install -q neo4j

     |████████████████████████████████| 90 kB 1.0 MB/s 


In [ ]:
from neo4j import GraphDatabase
import time
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 0)

In [ ]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [ ]:
uri  = 'bolt://3.86.237.233:7687' 
pwd  = 'arrow-briefing-crashes'
user = 'neo4j'
conn = Neo4jConnection(uri=uri, user=user , pwd=pwd)

In [ ]:
# creating constraints on graph

conn.query("CREATE CONSTRAINT BorderPoints IF NOT EXISTS ON (n:BorderPoint) ASSERT n.id IS UNIQUE;")
conn.query("CREATE CONSTRAINT Compressors IF NOT EXISTS ON (n:Compressor) ASSERT n.id IS UNIQUE;")
conn.query("CREATE CONSTRAINT Consumers IF NOT EXISTS ON (n:Consumer) ASSERT n.id IS UNIQUE;")
conn.query("CREATE CONSTRAINT LNGs IF NOT EXISTS ON (n:LNG) ASSERT n.id IS UNIQUE;")
conn.query("CREATE CONSTRAINT Nodes IF NOT EXISTS ON (n:Node) ASSERT n.id IS UNIQUE;")
conn.query("CREATE CONSTRAINT PowerPlants IF NOT EXISTS ON (n:PowerPlant) ASSERT n.id IS UNIQUE;")
conn.query("CREATE CONSTRAINT Productions IF NOT EXISTS ON (n:Production) ASSERT n.id IS UNIQUE;")
conn.query("CREATE CONSTRAINT Storages IF NOT EXISTS ON (n:Storage) ASSERT n.id IS UNIQUE;")

[]

## Component Creation

In [ ]:
# border points

conn.query("""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_BorderPoints.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     replace(row.name, '"','') as name,
     row.country_code as country_code,
     apoc.convert.fromJsonList(row.source_id) as source_ids,
     apoc.convert.fromJsonList(row.node_id) as node_ids,
     point({latitude:toFloat(row.lat), longitude:toFloat(row.long)}) as loc,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MERGE(b:BorderPoint {id:id})
ON CREATE SET b.name          = name,
              b.country_code  = country_code,
              b.source_id     = source_ids[0],
              b.node_id       = node_ids[0],
              b.loc           = loc,
              b              += params
""")

[]

In [ ]:
# compressors

conn.query("""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_Compressors.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     replace(row.name, '"','') as name,
     row.country_code as country_code,
     apoc.convert.fromJsonList(row.source_id) as source_ids,
     apoc.convert.fromJsonList(row.node_id) as node_ids,
     point({latitude:toFloat(row.lat), longitude:toFloat(row.long)}) as loc,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MERGE(c:Compressor {id:id})
ON CREATE SET c.name          = name,
              c.country_code  = country_code,
              c.source_id     = source_ids[0],
              c.node_id       = node_ids[0],
              c.loc           = loc,
              c              += params
""")

[]

In [ ]:
# consumers

conn.query("""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_Consumers.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     replace(row.name, '"','') as name,
     row.country_code as country_code,
     apoc.convert.fromJsonList(row.source_id) as source_ids,
     apoc.convert.fromJsonList(row.node_id) as node_ids,
     point({latitude:toFloat(row.lat), longitude:toFloat(row.long)}) as loc,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MERGE(c:Consumer {id:id})
ON CREATE SET c.name          = name,
              c.country_code  = country_code,
              c.source_id     = source_ids[0],
              c.node_id       = node_ids[0],
              c.loc           = loc,
              c              += params
""")

[]

In [ ]:
# LNGs

conn.query("""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_LNGs.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     replace(row.name, '"','') as name,
     row.country_code as country_code,
     apoc.convert.fromJsonList(row.source_id) as source_ids,
     apoc.convert.fromJsonList(row.node_id) as node_ids,
     point({latitude:toFloat(row.lat), longitude:toFloat(row.long)}) as loc,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MERGE(l:LNG {id:id})
ON CREATE SET l.name          = name,
              l.country_code  = country_code,
              l.source_id     = source_ids,
              l.node_id       = node_ids[0],
              l.loc           = loc,
              l              += params
""")

[]

In [ ]:
# storages

conn.query("""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_Storages.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     replace(row.name, '"','') as name,
     row.country_code as country_code,
     apoc.convert.fromJsonList(row.source_id) as source_ids,
     apoc.convert.fromJsonList(row.node_id) as node_ids,
     point({latitude:toFloat(row.lat), longitude:toFloat(row.long)}) as loc,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MERGE(s:Storage {id:id})
ON CREATE SET s.name          = name,
              s.country_code  = country_code,
              s.source_id     = source_ids,
              s.node_id       = node_ids[0],
              s.loc           = loc,
              s              += params
""")

[]

In [ ]:
# productions

conn.query("""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_Productions.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     replace(row.name, '"','') as name,
     row.country_code as country_code,
     apoc.convert.fromJsonList(row.source_id) as source_ids,
     apoc.convert.fromJsonList(row.node_id) as node_ids,
     point({latitude:toFloat(row.lat), longitude:toFloat(row.long)}) as loc,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MERGE(p:Production {id:id})
ON CREATE SET p.name          = name,
              p.country_code  = country_code,
              p.source_id     = source_ids[0],
              p.node_id       = node_ids[0],
              p.loc           = loc,
              p              += params
""")

[]

In [ ]:
# power plants

conn.query("""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_PowerPlants.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     replace(row.name, '"','') as name,
     row.country_code as country_code,
     apoc.convert.fromJsonList(row.source_id) as source_ids,
     apoc.convert.fromJsonList(row.node_id) as node_ids,
     point({latitude:toFloat(row.lat), longitude:toFloat(row.long)}) as loc,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MERGE(pp:PowerPlant {id:id})
ON CREATE SET pp.name          = name,
              pp.country_code  = country_code,
              pp.source_id     = source_ids,
              pp.node_id       = node_ids[0],
              pp.loc           = loc,
              pp              += params
""")

[]

In [ ]:
# nodes

conn.query("""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_Nodes.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     replace(row.name, '"','') as name,
     row.country_code as country_code,
     apoc.convert.fromJsonList(row.source_id) as source_ids,
     apoc.convert.fromJsonList(row.node_id) as node_ids,
     point({latitude:toFloat(row.lat), longitude:toFloat(row.long)}) as loc,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MERGE(n:Node {id:id})
ON CREATE SET n.name          = name,
              n.country_code  = country_code,
              n.source_id     = source_ids[0],
              n.node_id       = node_ids[0],
              n.loc           = loc,
              n              += params
""")

[]

In [ ]:
# pipe segments

conn.query("""
CALL apoc.periodic.iterate("
      LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_PipeSegments.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     row.name as name,
     apoc.convert.fromJsonList(row.country_code) as countries,
     apoc.convert.fromJsonList(row.node_id) as nodes,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MATCH (start_p:Node {id:nodes[0]})
MATCH (end_p:Node {id:nodes[1]})
RETURN start_p, end_p, id, name, params, countries
","   
      MERGE (start_p)-[pipe:PIPE {id:id}]->(end_p)
      ON CREATE SET pipe.name = name, 
                    pipe += params
", {batchSize:5, parallel:true}
)
""")

## EDA

In [ ]:
q = """
CALL db.labels() YIELD label
CALL apoc.cypher.run('MATCH (:`'+label+'`) RETURN count(*) as count',{}) YIELD value
RETURN label as nodes, value.count as nodeCount
ORDER BY nodeCount DESC
"""

df = pd.DataFrame([dict(_) for _ in conn.query(q)])
df

,nodes,nodeCount
0,Node,5009
1,Consumer,1357
2,PowerPlant,310
3,Storage,297
4,Compressor,248
5,BorderPoint,109
6,Production,104
7,LNG,32


In [ ]:
q = '''
CALL db.relationshipTypes() YIELD relationshipType as type
CALL apoc.cypher.run('MATCH ()-[:`'+type+'`]->() RETURN count(*) as count',{}) YIELD value
RETURN type as relationship, value.count as relationshipCount
'''
df = pd.DataFrame([dict(_) for _ in conn.query(q)])
df

,relationship,relationshipCount
0,PIPE,6526


In [ ]:
q = '''
CALL apoc.meta.stats()
'''
df = pd.DataFrame([dict(_) for _ in conn.query(q)])
df

,labelCount,relTypeCount,propertyKeyCount,nodeCount,relCount,labels,relTypes,relTypesCount,stats
0,8,1,83,7466,6526,"{'BorderPoint': 109, 'Storage': 297, 'Consumer': 1357, 'LNG': 32, 'PowerPlant': 310, 'Node': 5009, 'Production': 104, 'Compressor': 248}","{'()-[:PIPE]->(:Node)': 6526, '()-[:PIPE]->()': 6526, '(:Node)-[:PIPE]->()': 6526}",{'PIPE': 6526},"{'relTypeCount': 1, 'propertyKeyCount': 83, 'labelCount': 8, 'nodeCount': 7466, 'relCount': 6526, 'labels': {'BorderPoint': 109, 'Storage': 297, 'Consumer': 1357, 'LNG': 32, 'PowerPlant': 310, 'Node': 5009, 'Production': 104, 'Compressor': 248}, 'relTypes': {'()-[:PIPE]->(:Node)': 6526, '()-[:PIPE]->()': 6526, '(:Node)-[:PIPE]->()': 6526}}"


In [ ]:
query="""
MATCH p=(start:Node)-[:PIPE*20]->(next) 
WHERE not ((next)-[:PIPE]->()) AND start.id="INET_N_856"
RETURN DISTINCT [t in nodes(p) | t.id] as nodesPath, [t in nodes(p) | t.country_code] as countryPath
"""
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,nodesPath,countryPath
0,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, N_409_S_LMGN, SEQ_9701__M_LMGN, SEQ_9149__S_LMGN, SEQ_9139__S_LMGN, SEQ_9127__S_LMGN, N_486_M_LMGN, SEQ_7715__M_LMGN, SEQ_7723__M_LMGN, SEQ_7731__M_LMGN, SEQ_16065_p, SEQ_7733__L_LMGN, N_688_M_LMGN, SEQ_7739__L_LMGN, SEQ_7743__L_LMGN, SEQ_7747__L_LMGN, SEQ_7749__L_LMGN, NutsCons_1003]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL]"
1,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, N_409_S_LMGN, SEQ_9701__M_LMGN, SEQ_9149__S_LMGN, SEQ_9139__S_LMGN, SEQ_9127__S_LMGN, N_486_M_LMGN, SEQ_7715__M_LMGN, SEQ_7723__M_LMGN, SEQ_7731__M_LMGN, SEQ_16065_p, SEQ_7733__L_LMGN, SEQ_7731__L_LMGN, SEQ_7733__L_LMGN, N_688_M_LMGN, SEQ_7739__L_LMGN, SEQ_10595__M_LMGN, N_902_M_LMGN]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL]"
2,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, N_409_S_LMGN, SEQ_9701__M_LMGN, SEQ_9149__S_LMGN, SEQ_9139__S_LMGN, SEQ_9127__S_LMGN, N_486_M_LMGN, SEQ_7715__M_LMGN, SEQ_7723__M_LMGN, SEQ_7731__M_LMGN, SEQ_16065_p, SEQ_7733__L_LMGN, SEQ_7731__L_LMGN, SEQ_7727__L_LMGN, SEQ_7731__L_LMGN, SEQ_7733__L_LMGN, N_688_M_LMGN, N_902_M_LMGN]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL]"
3,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, N_409_S_LMGN, SEQ_9701__M_LMGN, N_1587_S_LMGN, SEQ_9693__M_LMGN, SEQ_9695__M_LMGN, SEQ_9701__M_LMGN, SEQ_9149__S_LMGN, SEQ_9139__S_LMGN, SEQ_9127__S_LMGN, N_486_M_LMGN, SEQ_7715__M_LMGN, SEQ_7723__M_LMGN, SEQ_7731__M_LMGN, N_688_M_LMGN, SEQ_7739__L_LMGN, SEQ_10595__M_LMGN, N_902_M_LMGN]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL]"
4,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, N_409_S_LMGN, SEQ_9701__M_LMGN, N_1587_S_LMGN, SEQ_9693__M_LMGN, SEQ_9695__M_LMGN, SEQ_9701__M_LMGN, SEQ_9149__S_LMGN, SEQ_9139__S_LMGN, SEQ_9127__S_LMGN, N_486_M_LMGN, SEQ_7715__M_LMGN, SEQ_7723__M_LMGN, SEQ_7731__M_LMGN, SEQ_16065_p, SEQ_7733__L_LMGN, N_688_M_LMGN, N_902_M_LMGN]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL]"
5,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, N_409_S_LMGN, SEQ_9701__M_LMGN, N_1587_S_LMGN, SEQ_9693__M_LMGN, SEQ_9695__M_LMGN, SEQ_9701__M_LMGN, SEQ_9149__S_LMGN, SEQ_9139__S_LMGN, SEQ_9127__S_LMGN, N_486_M_LMGN, SEQ_5209__M_LMGN, SEQ_6901_p, SEQ_5217__M_LMGN, SEQ_5231__M_LMGN, SEQ_5233__M_LMGN, SEQ_6861__S_LMGN, NutsCons_1165]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL]"
6,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, N_409_S_LMGN, SEQ_9701__M_LMGN, N_1587_S_LMGN, SEQ_9693__M_LMGN, SEQ_9695__M_LMGN, SEQ_8901_p, N_80_L_LMGN, SEQ_1693__L_LMGN, SEQ_1699__L_LMGN, SEQ_1703__L_LMGN, SEQ_1717__L_LMGN, N_81_L_LMGN, SEQ_7667__L_LMGN, N_417_S_LMGN, N_989_S_LMGN, SEQ_15267_p, NutsCons_1125]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL]"
7,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, N_409_S_LMGN, SEQ_9701__M_LMGN, N_1587_S_LMGN, SEQ_9693__M_LMGN, SEQ_9695__M_LMGN, SEQ_8901_p, N_80_L_LMGN, SEQ_1693__L_LMGN, SEQ_1699__L_LMGN, SEQ_1703__L_LMGN, SEQ_1717__L_LMGN, N_81_L_LMGN, SEQ_7667__L_LMGN, SEQ_7673__L_LMGN, SEQ_7675__L_LMGN, SEQ_7677__L_LMGN, INET_N_256]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL]"
8,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, N_409_S_LMGN, SEQ_9701__M_LMGN, N_1587_S_LMGN, SEQ_9693__M_LMGN, SEQ_9695__M_LMGN, SEQ_8901_p, N_80_L_LMGN, SEQ_1693__L_LMGN, SEQ_1699__L_LMGN, SEQ_1703__L_LMGN, SEQ_1717__L_LMGN, N_416_S_LMGN, SEQ_7667__L_LMGN, N_417_S_LMGN, N_989_S_LMGN, SEQ_15267_p, NutsCons_1125]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL, PL]"
9,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, N_409_S_LMGN, SEQ_9701__M_LMGN, N_1587_S_LMGN, SEQ_9693__M_LMGN, SEQ_9695__M_LMGN, SEQ_8901_p, N_80_L_LMGN, SEQ_1693__L_LMGN, SEQ_1699__L_LMGN, SEQ_1703__L_LMGN

## Graph Data Science

In [ ]:
!pip install -q graphdatascience

In [ ]:
import pandas as pd
from graphdatascience import GraphDataScience

uri  = 'bolt://3.86.237.233:7687' 
pwd  = 'arrow-briefing-crashes'
user = 'neo4j'
gds = GraphDataScience(uri, auth=("neo4j", pwd))

In [ ]:
# dropping an existing graph
query = "CALL gds.graph.drop('pipelines') YIELD graphName;"
gds.run_cypher(query)

,graphName
0,pipelines


In [ ]:
query = """CALL gds.graph.project(
                'pipelines',
                'Node',
                'PIPE'
            )
            YIELD
                graphName, nodeProjection, nodeCount, relationshipProjection, relationshipCount
"""

gds.run_cypher(query)

,graphName,nodeProjection,nodeCount,relationshipProjection,relationshipCount
0,pipelines,"{'Node': {'label': 'Node', 'properties': {}}}",5009,"{'PIPE': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'PIPE', 'properties': {}}}",6526


In [ ]:
G, res = gds.graph.project(
    'pipes',                 
    'Node',   
    "PIPE"                            
)
type(res)

pandas.core.series.Series

In [ ]:
print(res)

nodeProjection            {'Node': {'label': 'Node', 'properties': {}}}                                                   
relationshipProjection    {'PIPE': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'PIPE', 'properties': {}}}
graphName                 pipes                                                                                           
nodeCount                 5009                                                                                            
relationshipCount         6526                                                                                            
projectMillis             287                                                                                             
Name: 0, dtype: object


In [ ]:
G.node_count(), G.relationship_count()

(5009, 6526)

In [ ]:
G.degree_distribution()

p99     5.000000 
min     0.000000 
max     12.000000
mean    1.302855 
p90     3.000000 
p50     1.000000 
p999    7.000000 
p95     3.000000 
p75     2.000000 
dtype: float64

In [ ]:
res = gds.pageRank.stream(G)
res

,nodeId,score
0,2457,0.331688
1,2458,0.431934
2,2459,0.333572
3,2460,0.597695
4,2461,0.515261
...,...,...
5004,7461,0.411465
5005,7462,0.319784
5006,7463,0.267938
5007,7464,0.213750


In [ ]:
# top 10 by page rank
res.sort_values(by='score', ascending=False)[:10]

,nodeId,score
2848,5305,4.861119
2852,5309,4.331437
514,2971,4.225036
1813,4270,3.832889
1708,4165,3.320917
1769,4226,3.236406
1776,4233,3.128835
1709,4166,2.935067
2779,5236,2.765317
2112,4569,2.691821


In [ ]:
query = """
MATCH (n:Node)
WHERE ID(n) = 5305
RETURN n.id
"""
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,n.id
0,LKD_N_40


In [ ]:
# Centrality determined by node degree
query = """
MATCH (n:Node)
WITH n, SIZE((n)-[:PIPE]-(:Node)) AS degree
RETURN n.id AS id, n.country_code AS node_code, degree
ORDER BY degree DESC
"""
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,id,node_code,degree
0,LKD_N_38,DE,14
1,N_144_L_LMGN,GB,13
2,INET_N_1679,UA,11
3,N_72_L_LMGN,RU,11
4,LKD_N_147,DE,11
...,...,...,...
5004,NutsCons_307,CH,1
5005,NutsCons_200,ES,1
5006,NutsCons_235,CH,1
5007,NutsCons_588,GR,1


In [ ]:
# measure the nodes by indegree count
query = """MATCH (n:Node)
WITH n, SIZE((n)<-[:PIPE]-(:Node)) AS indegree
RETURN n.id AS id, n.country_code AS node_code, indegree
ORDER BY indegree DESC"""

df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,id,node_code,indegree
0,INET_N_1071,NL,7
1,INET_N_883,UA,7
2,LKD_N_40,DE,7
3,INET_N_408,UA,6
4,INET_N_1684,UA,6
...,...,...,...
5004,NutsCons_142,AL,0
5005,INET_N_1037,LT,0
5006,Prod_9,XX,0
5007,NutsCons_1261,GB,0


In [ ]:
# writing the page ranks of the nodes
q= """CALL gds.pageRank.write('pipes',
    {
        writeProperty: 'pagerank'
    }
)
YIELD nodePropertiesWritten, ranIterations"""
gds.run_cypher(q)

,nodePropertiesWritten,ranIterations
0,5009,20


In [ ]:
q = """MATCH (n:Node)<-[r:PIPE]-(:Node)
RETURN n.id AS id, n.country_code AS countryCode, round(n.pagerank,2) AS pagerank, round(sum(r.max_cap_M_m3_per_d),2) as maximumAnnualGasVolume
ORDER BY pagerank DESC, id ASC
LIMIT 10"""
df = pd.DataFrame([dict(_) for _ in conn.query(q)])
df

,id,countryCode,pagerank,maximumAnnualGasVolume
0,LKD_N_40,DE,4.86,195.52
1,LKD_N_44,DE,4.33,54.79
2,N_183_M_LMGN,MD,4.23,82.19
3,SEQ_1813__M_LMGN,MD,3.83,54.79
4,SEQ_7563__L_LMGN,XX,3.32,82.19
5,N_32_M_LMGN,RU,3.24,54.79
6,N_85_M_LMGN,RU,3.13,54.79
7,SEQ_7577__L_LMGN,XX,2.94,27.40
8,SEQ_2409_p,AZ,2.77,54.79
9,INET_N_408,UA,2.69,164.38


## Degree Centrality

In [ ]:
query = """
CALL gds.graph.project(
  'degreeCentralityPipes',
  'Node',
  {
    PIPE: {
      orientation: "UNDIRECTED"
    }
  }
)
"""
gds.run_cypher(query)

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'Node': {'label': 'Node', 'properties': {}}}","{'PIPE': {'orientation': 'UNDIRECTED', 'aggregation': 'DEFAULT', 'type': 'PIPE', 'properties': {}}}",degreeCentralityPipes,5009,13052,116


In [ ]:
query = """
CALL gds.degree.stream('degreeCentralityPipes')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS id, score AS degree
ORDER BY degree DESC, id ASC
LIMIT 10
"""
gds.run_cypher(query)

,id,degree
0,LKD_N_38,14.0
1,N_144_L_LMGN,13.0
2,INET_N_1679,11.0
3,INET_N_408,11.0
4,LKD_N_147,11.0
5,LKD_N_363,11.0
6,N_72_L_LMGN,11.0
7,INET_N_1071,10.0
8,INET_N_1118,10.0
9,INET_N_1579,10.0


## Betweenness Centrality

In [ ]:
query = "CALL gds.graph.project('betweennessPipes', 'Node', 'PIPE')"
gds.run_cypher(query)

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'Node': {'label': 'Node', 'properties': {}}}","{'PIPE': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'PIPE', 'properties': {}}}",betweennessPipes,5009,6526,249


In [ ]:
query = """
CALL gds.betweenness.write.estimate('betweennessPipes', { writeProperty: 'betweenness' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
"""

gds.run_cypher(query)

,nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
0,5009,6526,1404224,1404224,1371 KiB


In [ ]:
query = """
CALL gds.betweenness.stats('betweennessPipes')
YIELD centralityDistribution
RETURN centralityDistribution.min AS minimumScore, centralityDistribution.mean AS meanScore, centralityDistribution.max AS maxScore
"""
gds.run_cypher(query)

,minimumScore,meanScore,maxScore
0,0.0,17763.981371,864215.999992


In [ ]:
query = """
CALL gds.betweenness.stream('betweennessPipes')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS id, round(score,2) as score
ORDER BY score DESC, id ASC
LIMIT 10
"""

gds.run_cypher(query)

,id,score
0,INET_N_984,864212.91
1,LKD_N_38,607789.70
2,LKD_N_492,601340.02
3,INET_N_229,598424.32
4,SEQ_19574_p,598029.02
5,SEQ_19575_p,597328.02
6,LKD_N_491,597240.49
7,LKD_N_493,596627.02
8,SEQ_719__L_LMGN,595311.72
9,LKD_N_165,594963.55


In [ ]:
query = """
CALL gds.betweenness.stream('betweennessPipes')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS id, round(score,2) as score
ORDER BY score DESC, id ASC
"""

betweenness_df = gds.run_cypher(query)

In [ ]:
betweenness_df.head(20)

,id,score
0,INET_N_984,864212.91
1,LKD_N_38,607789.70
2,LKD_N_492,601340.02
3,INET_N_229,598424.32
4,SEQ_19574_p,598029.02
5,SEQ_19575_p,597328.02
6,LKD_N_491,597240.49
7,LKD_N_493,596627.02
8,SEQ_719__L_LMGN,595311.72
9,LKD_N_165,594963.55


## Louvain

In [ ]:
query = """
CALL gds.louvain.stream('pipes')
YIELD nodeId, communityId
RETURN
	communityId,
    SIZE(COLLECT(gds.util.asNode(nodeId).id)) AS number_of_nodes,
	COLLECT(gds.util.asNode(nodeId).id) AS ids,
    COLLECT(gds.util.asNode(nodeId).country_code) AS country_codes
ORDER BY number_of_nodes DESC, communityId;
"""
clusters_df = gds.run_cypher(query)
clusters_df.head(10)

,communityId,number_of_nodes,ids,country_codes
0,2691,211,"[SEQ_3791__L_LMGN, SEQ_3793__L_LMGN, N_1604_S_LMGN, SEQ_1229__L_LMGN, SEQ_9097__M_LMGN, N_2585_M_LMGN, SEQ_1235__L_LMGN, SEQ_5169__M_LMGN, SEQ_1237__L_LMGN, N_472_M_LMGN, N_594_M_LMGN, SEQ_201__L_LMGN, SEQ_199__L_LMGN, SEQ_145__L_LMGN, SEQ_195__L_LMGN, N_388_M_LMGN, SEQ_309__L_LMGN, SEQ_325__L_LMGN, SEQ_331__L_LMGN, N_390_M_LMGN, N_16_L_LMGN, SEQ_369__L_LMGN, SEQ_373__L_LMGN, SEQ_403__L_LMGN, SEQ_437__L_LMGN, SEQ_449__L_LMGN, SEQ_457__L_LMGN, SEQ_467__L_LMGN, N_17_L_LMGN, N_18_L_LMGN, SEQ_475__L_LMGN, SEQ_511__L_LMGN, SEQ_525__L_LMGN, SEQ_527__L_LMGN, SEQ_537__L_LMGN, SEQ_583__L_LMGN, SEQ_591__L_LMGN, N_19_L_LMGN, N_20_L_LMGN, N_21_L_LMGN, N_22_L_LMGN, SEQ_623__L_LMGN, SEQ_639__L_LMGN, N_24_L_LMGN, N_25_L_LMGN, N_26_L_LMGN, SEQ_805__L_LMGN, SEQ_1255__L_LMGN, N_82_L_LMGN, SEQ_1903__M_LMGN, SEQ_1757__L_LMGN, SEQ_1765__L_LMGN, SEQ_1771__L_LMGN, SEQ_1773__L_LMGN, SEQ_1797__L_LMGN, N_84_L_LMGN, SEQ_1829__L_LMGN, SEQ_1831__L_LMGN, SEQ_1833__L_LMGN, SEQ_1835__L_LMGN, SEQ_1839__L_LMGN, SEQ_1845__L_LMGN, N_86_L_LMGN, Stor_NonEU_21, N_34_M_LMGN, SEQ_3103__L_LMGN, SEQ_9386_p, SEQ_3813__L_LMGN, SEQ_3817__L_LMGN, N_182_L_LMGN, N_183_L_LMGN, N_258_M_LMGN, SEQ_1179__M_LMGN, SEQ_5955__M_LMGN, SEQ_5151__M_LMGN, SEQ_4003__M_LMGN, SEQ_5979__M_LMGN, SEQ_2383__M_LMGN, SEQ_7527__M_LMGN, SEQ_2715__M_LMGN, SEQ_5971__M_LMGN, SEQ_353__M_LMGN, SEQ_9501__M_LMGN, SEQ_5963__M_LMGN, SEQ_5989__M_LMGN, N_123_M_LMGN, SEQ_6069__M_LMGN, N_32_M_LMGN, N_53_M_LMGN, N_54_M_LMGN, N_83_M_LMGN, N_85_M_LMGN, N_118_M_LMGN, N_53_L_LMGN, SEQ_1183__M_LMGN, N_122_M_LMGN, N_125_M_LMGN, SEQ_1857__M_LMGN, SEQ_1895__M_LMGN, SEQ_1897__M_LMGN, ...]","[BY, BY, BY, BY, UA, UA, BY, BY, BY, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, BY, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, BY, BY, BY, RU, RU, RU, RU, RU, BY, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, RU, ...]"
1,763,183,"[N_735_M_LMGN, N_736_M_LMGN, N_1843_S_LMGN, N_1166_S_LMGN, N_1167_S_LMGN, SEQ_13739__S_LMGN, SEQ_5193__L_LMGN, N_160_L_LMGN, N_815_M_LMGN, N_816_M_LMGN, N_553_M_LMGN, SEQ_13823__S_LMGN, N_822_M_LMGN, N_1494_S_LMGN, N_1848_S_LMGN, SEQ_13927__S_LMGN, N_555_M_LMGN, SEQ_1573_p, N_2106_S_LMGN, N_2107_S_LMGN, SEQ_5149__L_LMGN, SEQ_5153__L_LMGN, SEQ_5165__L_LMGN, N_896_M_LMGN, SEQ_10731__M_LMGN, SEQ_10735__M_LMGN, INET_N_1481, SEQ_8163__M_LMGN, NutsCons_1073, SEQ_8159__M_LMGN, N_737_M_LMGN, N_556_M_LMGN, N_823_M_LMGN, N_897_M_LMGN, NutsCons_949, N_551_M_LMGN, SEQ_6581__M_LMGN, N_115_L_LMGN, N_113_L_LMGN, SEQ_2405_p, SEQ_16101__S_LMGN, SEQ_2557__L_LMGN, SEQ_2553_p, N_114_L_LMGN, SEQ_6681__L_LMGN, SEQ_5079__L_LMGN, SEQ_5077__L_LMGN, N_208_L_LMGN, SEQ_5129__L_LMGN, N_777_M_LMGN, SEQ_8955__M_LMGN, SEQ_3488_p, N_825_M_LMGN, N_1479_S_LMGN, SEQ_16009__S_LMGN, SEQ_3534_p, N_1828_S_LMGN, N_624_L_LMGN, SEQ_5179__L_LMGN, SEQ_5175__L_LMGN, SEQ_9273__M_LMGN, N_2108_S_LMGN, SEQ_17879__S_LMGN, N_1505_S_LMGN, SEQ_11007__S_LMGN, N_1169_S_LMGN, SEQ_5205__L_LMGN, SEQ_5207__L_LMGN, NutsCons_976, INET_N_219, N_914_S_LMGN, N_513_S_LMGN, N_1474_S_LMGN, N_1168_S_LMGN, N_1829_S_LMGN, NutsCons_1121, N_199_M_LMGN, N_200_M_LMGN, SEQ_7529_p, SEQ_8927__M_LMGN, SEQ_8933__M_LMGN, NutsCons_1081, SEQ_5183__L_LMGN, N_1478_S_LMGN, SEQ_7616_p, N_702_M_LMGN, N_850_M_LMGN, N_512_S_LMGN, N_923_S_LMGN, SEQ_5849__M_LMGN, N_510_S_LMGN, SEQ_8329_p, N_66_L_LMGN, SEQ_1493__L_LMGN, SEQ_1513__L_LMGN, SEQ_8865_p, SEQ_9678_p, SEQ_11107__S_LMGN, N_432_M_LMGN, N_687_S_LMGN, ...]","[IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, CH, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, IT, I

In [ ]:
clusters_df.tail(10)

,communityId,number_of_nodes,ids,country_codes
224,4662,1,[INET_N_118],[DE]
225,4675,1,[INET_N_1287],[IT]
226,4685,1,[INET_N_214],[DE]
227,4701,1,[INET_N_636],[DE]
228,4710,1,[INET_N_947],[DE]
229,4740,1,[INET_N_270],[GB]
230,4922,1,[INET_N_900],[SK]
231,4929,1,[NutsCons_1515],[XX]
232,4956,1,[NutsCons_597],[XX]
233,4973,1,[INET_N_522],[XX]


## Path Finding

In [ ]:
query = """
CALL gds.graph.project(
    'routes-weighted',
    'Node',
    'PIPE',
        {
            relationshipProperties: 'length_km'
        }
)
"""

gds.run_cypher(query)

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'Node': {'label': 'Node', 'properties': {}}}","{'PIPE': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'PIPE', 'properties': {'length_km': {'defaultValue': None, 'property': 'length_km', 'aggregation': 'DEFAULT'}}}}",routes-weighted,5009,6526,495


In [ ]:
query = """
MATCH (source:Node {id: 'INET_N_856'}), (target:Node {id: 'NutsCons_1003'})
CALL gds.shortestPath.dijkstra.stream('routes-weighted', {
    sourceNode: source,
    targetNode: target,
    relationshipWeightProperty: 'length_km'
})
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    index,
    gds.util.asNode(sourceNode).id AS sourceNodeName,
    gds.util.asNode(targetNode).id AS targetNodeName,
    totalCost as totalDistance,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).id] AS nodeIDs,
    costs as distances
ORDER BY index
"""
gds.run_cypher(query)

,index,sourceNodeName,targetNodeName,totalDistance,nodeIDs,distances
0,0,INET_N_856,NutsCons_1003,649.707706,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, N_409_S_LMGN, SEQ_9701__M_LMGN, SEQ_9149__S_LMGN, SEQ_9139__S_LMGN, SEQ_9127__S_LMGN, N_486_M_LMGN, SEQ_7715__M_LMGN, SEQ_7723__M_LMGN, SEQ_7731__M_LMGN, N_688_M_LMGN, SEQ_7739__L_LMGN, SEQ_7743__L_LMGN, SEQ_7747__L_LMGN, SEQ_7749__L_LMGN, NutsCons_1003]","[0.0, 55.420951, 110.841205, 220.16349100000002, 253.03525000000002, 260.771973, 289.15599, 317.347467, 352.397237, 372.523689, 398.64748099999997, 448.01613399999997, 487.93127499999997, 540.4601309999999, 569.220877, 597.2742549999999, 630.323322, 638.421699, 649.707706]"
